In [1]:
import pandas as pd
import utils
import matplotlib.pyplot as plt
import seaborn as sns

## DATASET

In [2]:
Test = pd.read_csv("../data/test_data.csv")
Transactional = pd.read_csv("../data/transactional_data.csv", sep=';')
Geocode = pd.read_csv("../data/df_geocode.csv")

### GEOCODE

In [3]:
Geocode.head()

,Unnamed: 0,Lab Id,Address,Location,Zipcode
0,0,L152,"3800 PLEASANT HILL RD STE 1, DULUTH, GA 30096","34.000319,-84.1629724",30096.0
1,1,L520,"1614 N JAMES ST, ROME, NY 13440","43.2311327,-75.4445363",13440.0
2,2,L141,"12911 120TH AVE NE STE D60, KIRKLAND, WA 98034","47.7162786,-122.1838152",98034.0
3,3,L524,"5667 PEACHTREE DUNWOODY RD 250, ATLANTA, GA 30342","33.9093875,-84.3529096",30342.0
4,4,L545,"1204 IL HWY 164, OQUAWKA, IL 61469","40.9309925,-90.9437598",61469.0


In [4]:
Geocode = Geocode.drop(columns='Unnamed: 0')

In [5]:
Geocode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Lab Id    119 non-null    object 
 1   Address   119 non-null    object 
 2   Location  119 non-null    object 
 3   Zipcode   118 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.8+ KB


In [6]:
for col in Geocode.columns:
    print(f"{col} {len(Geocode[col].unique())}")

Lab Id 119
Address 119
Location 117
Zipcode 106


### TRANSACTIONAL

In [7]:
Transactional['Date of birth'] = pd.to_datetime(Transactional['Date of birth'], format='%d/%m/%Y %H:%M:%S')
Transactional['Date of service'] = pd.to_datetime(Transactional['Date of service'])
Transactional['Testing Cost'] = Transactional['Testing Cost'].str.replace(',', '.')
Transactional['Testing Cost'] = pd.to_numeric(Transactional['Testing Cost'])

In [8]:
# Mudança de nome dado ao enunciado
Transactional.rename(columns={'Testing Cost': 'Revenue'}, inplace=True)

In [9]:
Transactional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2355241 entries, 0 to 2355240
Data columns (total 8 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Patient Id       object        
 1   Gender           object        
 2   Date of birth    datetime64[ns]
 3   Date of service  datetime64[ns]
 4   Service Id       object        
 5   Lab Id           object        
 6   CodItem          int64         
 7   Revenue          float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 143.8+ MB


### TEST

In [10]:
Test.head()

,CodItem,Desc Item,Category,Family,Speciality,Testing Cost
0,70003237,"MAGNESIO, SORO (EXEMPLO DE EXPRESSAO DE RESULT...",CA,CORELAB,BIOCHEMISTRY,1.78
1,70000638,"HEMOGRAMA, AUTOMATIZADO, SANGUE",CA,HEMATO,BLOOD COUNT,2.46
2,70001597,"FERRITINA, SORO",CA,CORELAB,IMMUNOHORMONE,2.11
3,70000103,"FERRO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE ...",CA,CORELAB,BIOCHEMISTRY,0.80
4,70000224,"CALCIO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE...",CA,CORELAB,BIOCHEMISTRY,1.02


In [11]:
for col in Test.columns:
    print(f"{col} {len(Test[col].unique())}")

CodItem 2001
Desc Item 1997
Category 7
Family 23
Speciality 77
Testing Cost 1670


In [12]:
# CodItem 70004701 estava duplicado e sua categoria estava preenchida com ' '.
Test['Category'] = Test['Category'].str.replace(' ', '')

In [13]:
Test = Test.drop_duplicates()

In [14]:
Test['CodItem'] = pd.to_numeric(Test['CodItem'])

In [15]:
Test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2001 entries, 0 to 2001
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CodItem       2001 non-null   int64  
 1   Desc Item     2001 non-null   object 
 2   Category      2001 non-null   object 
 3   Family        2001 non-null   object 
 4   Speciality    2001 non-null   object 
 5   Testing Cost  2001 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 109.4+ KB


### MERGE DATASETS

In [16]:
df = pd.merge(Transactional, Test, how="left", on="CodItem")
print(f"Lines: Transactional {len(Transactional['CodItem'])}, Merged {len(df['CodItem'])}, diferença {len(Transactional['CodItem']) - len(df['CodItem'])} ")
df.head()

Lines: Transactional 2355241, Merged 2355241, diferença 0 


,Patient Id,Gender,Date of birth,Date of service,Service Id,Lab Id,CodItem,Revenue,Desc Item,Category,Family,Speciality,Testing Cost
0,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70003237,9.0,"MAGNESIO, SORO (EXEMPLO DE EXPRESSAO DE RESULT...",CA,CORELAB,BIOCHEMISTRY,1.78
1,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70000638,13.0,"HEMOGRAMA, AUTOMATIZADO, SANGUE",CA,HEMATO,BLOOD COUNT,2.46
2,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70001597,49.0,"FERRITINA, SORO",CA,CORELAB,IMMUNOHORMONE,2.11
3,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70000103,11.0,"FERRO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE ...",CA,CORELAB,BIOCHEMISTRY,0.80
4,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70000224,10.0,"CALCIO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE...",CA,CORELAB,BIOCHEMISTRY,1.02


In [17]:
df = pd.merge(df, Geocode, how="left", on="Lab Id")
print(f"Lines: Transactional {len(Transactional['CodItem'])}, Merged {len(df['CodItem'])}, diferença {len(Transactional['CodItem']) - len(df['CodItem'])} ")
df.head()

Lines: Transactional 2355241, Merged 2355241, diferença 0 


,Patient Id,Gender,Date of birth,Date of service,Service Id,Lab Id,CodItem,Revenue,Desc Item,Category,Family,Speciality,Testing Cost,Address,Location,Zipcode
0,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70003237,9.0,"MAGNESIO, SORO (EXEMPLO DE EXPRESSAO DE RESULT...",CA,CORELAB,BIOCHEMISTRY,1.78,"2220 GLADSTONE DR. SUITE 7, PITTSBURG, CA 94565","38.0070008,-121.868574",94565.0
1,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70000638,13.0,"HEMOGRAMA, AUTOMATIZADO, SANGUE",CA,HEMATO,BLOOD COUNT,2.46,"2220 GLADSTONE DR. SUITE 7, PITTSBURG, CA 94565","38.0070008,-121.868574",94565.0
2,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70001597,49.0,"FERRITINA, SORO",CA,CORELAB,IMMUNOHORMONE,2.11,"2220 GLADSTONE DR. SUITE 7, PITTSBURG, CA 94565","38.0070008,-121.868574",94565.0
3,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70000103,11.0,"FERRO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE ...",CA,CORELAB,BIOCHEMISTRY,0.80,"2220 GLADSTONE DR. SUITE 7, PITTSBURG, CA 94565","38.0070008,-121.868574",94565.0
4,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70000224,10.0,"CALCIO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE...",CA,CORELAB,BIOCHEMISTRY,1.02,"2220 GLADSTONE DR. SUITE 7, PITTSBURG, CA 94565","38.0070008,-121.868574",94565.0


### NEW COLUMNS

In [18]:
df['MargemBruta'] = df['Revenue'] - df['Testing Cost']
df['rentabilidade'] = df['MargemBruta'] / df['Testing Cost']

In [19]:
df.head()

,Patient Id,Gender,Date of birth,Date of service,Service Id,Lab Id,CodItem,Revenue,Desc Item,Category,Family,Speciality,Testing Cost,Address,Location,Zipcode,MargemBruta,rentabilidade
0,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70003237,9.0,"MAGNESIO, SORO (EXEMPLO DE EXPRESSAO DE RESULT...",CA,CORELAB,BIOCHEMISTRY,1.78,"2220 GLADSTONE DR. SUITE 7, PITTSBURG, CA 94565","38.0070008,-121.868574",94565.0,7.22,4.056180
1,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70000638,13.0,"HEMOGRAMA, AUTOMATIZADO, SANGUE",CA,HEMATO,BLOOD COUNT,2.46,"2220 GLADSTONE DR. SUITE 7, PITTSBURG, CA 94565","38.0070008,-121.868574",94565.0,10.54,4.284553
2,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70001597,49.0,"FERRITINA, SORO",CA,CORELAB,IMMUNOHORMONE,2.11,"2220 GLADSTONE DR. SUITE 7, PITTSBURG, CA 94565","38.0070008,-121.868574",94565.0,46.89,22.222749
3,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70000103,11.0,"FERRO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE ...",CA,CORELAB,BIOCHEMISTRY,0.80,"2220 GLADSTONE DR. SUITE 7, PITTSBURG, CA 94565","38.0070008,-121.868574",94565.0,10.20,12.750000
4,10210830256-BIO003,F,1976-08-01,2019-01-07,571904533475-38,L133,70000224,10.0,"CALCIO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE...",CA,CORELAB,BIOCHEMISTRY,1.02,"2220 GLADSTONE DR. SUITE 7, PITTSBURG, CA 94565","38.0070008,-121.868574",94565.0,8.98,8.803922


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2355241 entries, 0 to 2355240
Data columns (total 18 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Patient Id       object        
 1   Gender           object        
 2   Date of birth    datetime64[ns]
 3   Date of service  datetime64[ns]
 4   Service Id       object        
 5   Lab Id           object        
 6   CodItem          int64         
 7   Revenue          float64       
 8   Desc Item        object        
 9   Category         object        
 10  Family           object        
 11  Speciality       object        
 12  Testing Cost     float64       
 13  Address          object        
 14  Location         object        
 15  Zipcode          float64       
 16  MargemBruta      float64       
 17  rentabilidade    float64       
dtypes: datetime64[ns](2), float64(5), int64(1), object(10)
memory usage: 323.4+ MB


### SAVE

In [21]:
df.to_csv("../data/Exams_cleaned.csv")